In [1]:
import os
from binance_logger import CryptoMarket,DB
import logging
db = DB()
market = CryptoMarket()
market.update_pairs_klines()
market.record_kline_data(db,"crypto_market")

In [2]:
market.update_pairs_klines()
market.record_kline_data(db,"crypto_market")

In [6]:
def record_kline_data(self,db,bucket_name):
    points = []
    for pair in self.pairs.values():
        if isinstance(pair.kline,list) and len(pair.kline)>0:
            pair_dict = pair.dict()
            tags = {key:val for key,val in pair_dict.items() if key not in ["price","kline"]}
            for k in pair.kline:
                point = {
                    "measurement":"kline",
                    "tags":tags,
                    "fields":{key:val for key,val in k.dict().items() if key not in ["openTime","closeTime"]},
                    "time":k.closeTime
                }
                points.append(point)
    db._write_client.write(bucket_name,db.org,points,'ms')

record_kline_data(market,db,"crypto_market")

In [15]:
query="""from(bucket: "crypto_market")
  |> range(start: -45m)
  |> filter(fn: (r) => r["_measurement"] == "kline")
  |> filter(fn: (r) => r["symbol"] == "AAVEUPUSDT")
  |> last()"""
query_result = db.query_api.query(query)
times = []
for table in query_result:
    for entry in table:
        times.append(entry.get_time())
dtb = max(times)
print(dtb)

2021-03-21 01:38:59.999000+00:00


In [16]:
from datetime import datetime
from datetime import timezone

dt = datetime.fromtimestamp(market.pairs.get("AAVEUPUSDT").kline[-1].closeTime/1000)
# dt.replace(tzinfo=timezone.utc)
print(dt)

2021-03-21 02:15:59.999000


In [19]:
38+2220/60

75.0

In [17]:
dtb.timestamp()-dt.timestamp()

-2220.0

In [14]:
dt.timestamp()

1616292959.999